<a href="https://colab.research.google.com/github/sodonne6/Data_Augmentation_for_AVSR/blob/main/Av_Hubert_first_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
%pip -q install -U condacolab
import condacolab
condacolab.install() #restarts runtime

✨🍰✨ Everything looks OK!


In [29]:
#create and activate python 3.8 env
%%bash
eval "$(conda shell.bash hook)"
conda env remove -n av_hubert || true # Remove the environment if it exists
conda create -n av_hubert python=3.8
conda activate av_hubert

# clone and install dependencies inside env
git clone https://github.com/facebookresearch/av_hubert.git /content/av_hubert
cd /content/av_hubert
git submodule init
git submodule update

#show content for sanity check
pwd
ls -lah


Remove all packages in environment /usr/local/envs/av_hubert:


## Package Plan ##

  environment location: /usr/local/envs/av_hubert


The following packages will be REMOVED:

  _libgcc_mutex-0.1-conda_forge
  _openmp_mutex-4.5-2_gnu
  bzip2-1.0.8-hda65f42_8
  ca-certificates-2025.8.3-hbd8a1cb_0
  ld_impl_linux-64-2.44-h1423503_1
  libffi-3.4.6-h2dba641_1
  libgcc-15.1.0-h767d61c_5
  libgcc-ng-15.1.0-h69a702a_5
  libgomp-15.1.0-h767d61c_5
  liblzma-5.8.1-hb9d3cd8_2
  liblzma-devel-5.8.1-hb9d3cd8_2
  libnsl-2.0.1-hb9d3cd8_1
  libsqlite-3.50.4-h0c1763c_0
  libuuid-2.41.2-he9a06e4_0
  libxcrypt-4.4.36-hd590300_1
  libzlib-1.3.1-hb9d3cd8_2
  ncurses-6.5-h2d0b736_3
  openssl-3.5.3-h26f9b46_1
  pip-24.3.1-pyh8b19718_0
  python-3.8.20-h4a871b0_2_cpython
  readline-8.2-h8c095d6_2
  setuptools-75.3.0-pyhd8ed1ab_0
  tk-8.6.13-noxft_hd72426e_102
  wheel-0.45.1-pyhd8ed1ab_0
  xz-5.8.1-hbcc6ac9_2
  xz-gpl-tools-5.8.1-hbcc6ac9_2
  xz-tools-5.8.1-hb9d3cd8_2



Preparing transaction: - done
Verify



==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda


fatal: destination path '/content/av_hubert' already exists and is not an empty directory.


In [30]:
# install deps inside the conda env
%%bash
eval "$(conda shell.bash hook)"
conda activate av_hubert
cd /content/av_hubert/

# 0) Repo requirements
pip install -r requirements.txt

# 1) Pin NumPy so np.float exists
pip install --no-cache-dir "numpy==1.23.5"

# 2) Pin config stack that works with this fairseq tree
pip uninstall -y omegaconf hydra-core || true
pip install --no-cache-dir omegaconf==2.1.1 hydra-core==1.1.0

# 3) Torch (GPU build via pip cu117 wheels)
conda remove -y pytorch torchvision torchaudio pytorch-cuda || true
pip uninstall -y torch torchvision torchaudio || true
pip install --no-cache-dir \
  torch==1.13.1+cu117 \
  torchvision==0.14.1+cu117 \
  torchaudio==0.13.1 \
  --extra-index-url https://download.pytorch.org/whl/cu117

# 4) Install fairseq from the bundled submodule (editable) **without deps**
cd fairseq
pip install -e . --no-deps
cd ..

# 5) Re-assert the pins in case anything tried to downgrade them (belt & suspenders)
pip install --no-cache-dir --upgrade --upgrade-strategy only-if-needed \
  omegaconf==2.1.1 hydra-core==1.1.0

# 6) Tiny import check
python - << 'PY'
import torch, fairseq, numpy as np
from omegaconf import II, MISSING
import hydra
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())
print("Fairseq import OK")
print("NumPy:", np.__version__)
print("OmegaConf II/MISSING present:", II is not None and MISSING is not None)
print("Hydra:", hydra.__version__)
PY


  Using cached python_speech_features-0.6-py3-none-any.whl
  Using cached scipy-1.10.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
  Using cached opencv_python-4.5.4.60-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached sentencepiece-0.1.96-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached editdistance-0.6.0-cp38-cp38-manylinux2010_x86_64.whl.metadata (4.7 kB)
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached scipy-1.10.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
Using cached opencv_python-4.5.4.60-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.3 MB)
Using cached sentencepiece-0.1.96-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
Using cached editdistance-0.6.0-cp38-cp38-manylinux2010_x86_64.whl (286 kB)
Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_


PackagesNotFoundError: The following packages are missing from the target environment:
  - torchaudio
  - pytorch
  - torchvision
  - pytorch-cuda




In [33]:
#CODE BEGINNING -> MOUNT GOOGLE DRIVE
#NEEDS TO BE DONE EVERYTIME

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
#choose a model to use
model1_name = "base_vox_iter5.pt"
model2_name = "base_noise_pt_noise_ft_30h.pt"

## Load in the pretrained model


In [35]:
%%bash
eval "$(conda shell.bash hook)"
conda activate av_hubert
cd /content  # stay outside the repo root

# Put the avhubert *subfolder* on PYTHONPATH so top-level imports work
PYTHONPATH=/content/av_hubert/avhubert:$PYTHONPATH \
python -u - << 'PY'
import os, torch, importlib

# 1) Import fairseq first
import fairseq
from fairseq.tasks import TASK_REGISTRY
print("fairseq OK; registry size before:", len(TASK_REGISTRY))

# 2) Force-import the files that perform registration
#    (these use @register_task / @register_model decorators)
importlib.import_module("hubert_pretraining")  # registers 'av_hubert_pretraining'
importlib.import_module("hubert")              # registers the AV-HuBERT model

# 3) Verify registration
from fairseq.tasks import TASK_REGISTRY
print("Contains 'av_hubert_pretraining':", "av_hubert_pretraining" in TASK_REGISTRY)

# 4) Load checkpoint
from fairseq.checkpoint_utils import load_model_ensemble_and_task
ckpt_path = "/content/drive/MyDrive/AVSR_Colab/models/base_vox_iter5.pt"  # adjust if needed
assert os.path.exists(ckpt_path), f"Checkpoint not found: {ckpt_path}"

models, cfg, task = load_model_ensemble_and_task([ckpt_path])
model = models[0]

print("✅ Loaded AV-HuBERT checkpoint")
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("Model:", type(model).__name__, "| Task:", type(task).__name__)
PY


fairseq OK; registry size before: 25
Contains 'av_hubert_pretraining': True
2025-09-23 17:32:52 | INFO | hubert_pretraining | current directory is /content
2025-09-23 17:32:52 | INFO | hubert_pretraining | AVHubertPretrainingTask Config {'_name': 'av_hubert_pretraining', 'data': '/checkpoint/bshi/data/lrs3//video/wav/envox_tsv/', 'labels': ['km'], 'label_dir': '/checkpoint/bshi/data/lrs3//video/hubert/stitch-iters/envox-iter4-l12c2000/', 'label_rate': 25, 'sample_rate': 25, 'normalize': True, 'enable_padding': False, 'max_sample_size': 2000, 'min_sample_size': 5, 'max_trim_sample_size': 400, 'single_target': False, 'random_crop': True, 'pad_audio': False, 'pdb': False, 'stack_order_audio': 4, 'skip_verify': False, 'image_aug': True, 'image_crop_size': 88, 'image_mean': 0.421, 'image_std': 0.165, 'modalities': ['audio', 'video'], 'is_s2s': False, 'tokenizer_bpe_name': None, 'tokenizer_bpe_model': None, 'noise_wav': '/checkpoint/bshi/data/lrs3//audio/noise/tsv/musan-lgall/', 'noise_prob'

##Setup Directory Layout

In [36]:
%%bash
# Make a tidy layout on your Google Drive
mkdir -p "/content/drive/MyDrive/AVSR_Colab"/{data,manifests,experiments,models,logs}

# Inside 'experiments', make a folder for your first run
mkdir -p "/content/drive/MyDrive/AVSR_Colab/experiments/baseline"


In [45]:
# Point these at your Drive layout so later cells can reuse them
BASE = "/content/drive/MyDrive/AVSR_Colab"
DATA_DIR = f"{BASE}/data"              # where raw/augmented media will live
MANIFEST_DIR = f"{BASE}/manifests"     # where TSVs (indexes) will live
EXP_DIR = f"{BASE}/experiments"        # one subfolder per experiment
MODELS_DIR = f"{BASE}/models"          # checkpoints you already have (and new ones)
LOGS_DIR = f"{BASE}/logs"              # training logs / tensorboard etc.

print("BASE       :", BASE)
print("DATA_DIR   :", DATA_DIR)
print("MANIFESTS  :", MANIFEST_DIR)
print("EXPERIMENTS:", EXP_DIR)
print("MODELS_DIR :", MODELS_DIR)
print("LOGS_DIR   :", LOGS_DIR)


BASE       : /content/drive/MyDrive/AVSR_Colab
DATA_DIR   : /content/drive/MyDrive/AVSR_Colab/data
MANIFESTS  : /content/drive/MyDrive/AVSR_Colab/manifests
EXPERIMENTS: /content/drive/MyDrive/AVSR_Colab/experiments
MODELS_DIR : /content/drive/MyDrive/AVSR_Colab/models
LOGS_DIR   : /content/drive/MyDrive/AVSR_Colab/logs


In [38]:
%%bash
eval "$(conda shell.bash hook)"
conda activate av_hubert

python - << 'PY'
import torch, importlib
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())
try:
    import fairseq
    print("fairseq OK:", getattr(fairseq, "__file__", None))
except Exception as e:
    print("fairseq import issue:", e)

# config stack the repo expects
try:
    from omegaconf import II, MISSING
    import hydra
    print("OmegaConf II/MISSING present ✓ | Hydra:", hydra.__version__)
except Exception as e:
    print("Config stack issue:", e)
PY


Torch: 1.13.1+cu117 | CUDA available: False
fairseq OK: /content/av_hubert/fairseq/fairseq/__init__.py
OmegaConf II/MISSING present ✓ | Hydra: 1.1.0


In [48]:
%%bash
BASE="/content/drive/MyDrive/AVSR_Colab"
mkdir -p "$BASE/experiments/baseline"/{checkpoints,logs,manifests,configs} #create sub directories for baseline test
echo "Created scaffolds under $BASE/experiments/*"

Created scaffolds under /content/drive/MyDrive/AVSR_Colab/experiments/*


In [49]:
#freeze enironment (do this for each experiment so it can be replicated)
%%bash
eval "$(conda shell.bash hook)"
conda activate av_hubert

BASE="/content/drive/MyDrive/AVSR_Colab"
pip freeze > "$BASE/experiments/baseline/configs/pip_freeze.txt"
python - << 'PY' > "$BASE/experiments/baseline/configs/env_info.txt"
import torch, sys, platform
print("python:", sys.version)
print("platform:", platform.platform())
print("torch:", torch.__version__, "cuda_available:", torch.cuda.is_available())
PY
echo "Saved env snapshot."


Saved env snapshot.
